<a href="https://colab.research.google.com/github/sausterm/doggit/blob/master/scrape_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!pip install Pillow
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
import numpy as np
from glob import glob
import os
import io
from PIL import Image
import hashlib
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from google.colab import drive

     |████████████████████████████████| 911kB 4.2MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [42.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bioni

In [3]:
drive.mount('/content/drive')
os.chdir('drive/My Drive/Projects/DLND/dog_app')
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['scrape_images.ipynb',
 'haarcascades',
 'dogImages',
 'lfw',
 'test_imgs',
 'models',
 'model.pth',
 'model_res50.pth',
 'ChromeDriver',
 'images',
 'dog_app.ipynb',
 'model_dense.pth',
 'model_dev2.ipynb']

In [6]:
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get('https://google.com')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [10]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:wd, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [11]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
      pass
        #print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        #print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
      pass
        #print(f"ERROR - Could not save {url} - {e}")

In [12]:
def search_and_download(search_term:str,driver_path:str,target_path,number_images=5):
    #target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_path):
        os.makedirs(target_path)

    """with webdriver.Chrome(executable_path=driver_path) as wd:"""
    res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
    print('urls fetched.')
    for elem in res:
        persist_image(target_path,elem)
    print('uploaded {} images'.format(search_term))

In [13]:
DRIVER_PATH= "./ChromeDriver/chromedriver"
search_term = 'Rock'
target_path = "./test_scraped_images"

search_and_download(
    search_term = search_term,
    driver_path = DRIVER_PATH,
    target_path = target_path
)

Found: 100 search results. Extracting links from 0:100
Found: 5 image links, done!
urls fetched.
uploaded Rock images


In [ ]:
breeds = [] 

for i in range(len(dog_files_train)):
  breeds.append(str(dog_files_train[i]).split('/')[2][4:])
breeds = np.array(breeds)
breeds = np.unique(breeds)
breeds = list(breeds)

lst = [x for x in range(len(breeds))]
breeds = list(zip(lst,breeds))


In [ ]:
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [ ]:

for i, breed in breeds:
  if breed != 'Newfoundland':
    pass
  n = str(round((0.001*(i+1)),3))[2:]
  while len(n)<3:
    n = n+"0"
  f = "{}.{}".format(n,str(breed))

  target_path = "./dogImages/train/{}".format(f)
  search_term = "Newfoundland Dog"
  print("search term = {}".format(search_term))

  search_and_download(
    search_term = search_term,
    driver_path = DRIVER_PATH,
    target_path = target_path,
    number_images = 100
  )
  print(target_path)

search term = Newfoundland Dog


MaxRetryError: ignored